# Probabilisitic Error Cancellation with Mitiq

### In this section we will complete the following steps:
#### 1. Set up an example circuit in a Mitiq-supported frontend
#### 2. Step through the two main components of PEC: 
     a. generating quasiprobability representations
     b. sampling from the representations
#### 3. Use Mitiq's top level function `mitiq.pec.execute_with_pec()` and check that the results are equivalent to those obtained in Step 2.

## Step 1: Set up circuit and representations of gates

In [10]:
import mitiq
from mitiq import benchmarks

frontend = "cirq"  # Supported: "cirq", "qiskit", "pyquil", "braket", "pennylane".

circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend,
)[0]
print(circuit)

0: ───X───Y^0───Y───X^0───Y───X───


Define the list of `OperationRepresentation`s (one for each gate).

An `OperationRepresentation` is a decomposition (basis expansion) of an operation or sequence of operations in a basis of noisy, implementable operations.

For this example we assume local depolarizing noise.

In [11]:
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {noise_level :.2%} depolarizing noise.")

4 OperationRepresentation objects produced, assuming 1.00% depolarizing noise.


Let's look at the first `OperationRepresentation` in the list `reps`:

In [3]:
print(reps[0])

0: ───X^0.5─── = 1.010*(0: ───X^0.5───)-0.003*(0: ───X^0.5───X───)-0.003*(0: ───X^0.5───Y───)-0.003*(0: ───X^0.5───Z───)


## Step 2a: Probabilistic generation of all auxiliary circuits

The following steps are implemented in the Mitiq function sample_circuit() to probabilistically generate an integer number of auxiliary circuits, equivalent to the number of PEC samples:
- Define an empty `sampled_circuit` to be populated with probabilistic operations
- Define an empty `gate_sign_list` to be populated with the sign values
- Start a loop over the ideal operations of circuit:
    - Search for the corresponding `OperationRepresentation` in the input list of quasiprobability representations
    - Sample a noisy gate from the quasi-probability distribution of the ideal gate using `rep[j].sample()`
    - Append the sampled gate to `sampled_circuit` and the corresponding sign to `gate_sign_list`;
- Return `sampled_circuit` and  associated `sampled_sign`, i.e. the product of all the elements of `gate_sign_list`.

Let's check the first 5 circuits, signs, and the one-norm:

In [4]:
from mitiq import pec

sampled_circuits, sampled_signs, one_norm = pec.sample_circuit(circuit, reps, num_samples=200, random_state=30)
for circuit in sampled_circuits[:5]:
    print(circuit)
print("Signs:", sampled_signs[:5])
print("One-norm:", one_norm)

0: ───Y───X^-0.5───Y^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^-0.5───
0: ───Y───X^-0.5───Z───Y^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^-0.5───
0: ───Y───X^-0.5───Y^-0.5───X───X^0.5───Y^-0.5───X^-0.5───Y^-0.5───
0: ───Y───X^-0.5───Y^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^-0.5───
0: ───Y───X^-0.5───Y^-0.5───X^0.5───Y^-0.5───X^-0.5───Y^-0.5───
Signs: [1, -1, -1, 1, 1]
One-norm: 1.1508179395702227


## Step 2b: Inference of the ideal expectation value from the noisy execution of the auxiliary circuits

Define a batched executor, i.e. an executor function that takes a list of circuits as input and returns a list of expectation values.

In [5]:
from typing import List

from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def batched_execute(circuits: List[mitiq.QPROGRAM], noise_level: float=0.01)->List[float]:
    """Returns [Tr[ρ_1 |0⟩⟨0|], Tr[ρ_2 |0⟩⟨0|]... ] where ρ_j is the state prepared by the
    j_th circuit in the input argument "circuits".
    """
    # Replace with code based on your frontend and backend, possibly using a batched execution.
    expectation_values = []
    for circuit in circuits:
        mitiq_circuit, _ = convert_to_mitiq(circuit)
        noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
        rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
        expectation_values.append(rho[0, 0].real)
    return expectation_values

Execute all the auxiliary circuits generated in the previous section to obtain a list of noisy expectation values.

In [6]:
executor = mitiq.Executor(batched_execute, max_batch_size=100)
noisy_expecation_values = executor.evaluate(
    sampled_circuits, 
    force_run_all=False,
)

# Unique noisy expectation values associated to unique circuits in sampled_circuits
executor.quantum_results[0:10]

[0.9551591,
 0.05090978,
 0.050909836,
 0.05090978,
 0.05090978,
 0.05090979,
 0.94909036,
 0.050909836,
 0.9490902,
 0.050909836]

In [7]:
print(f"{len(noisy_expecation_values)} noisy expectation values efficiently evaluated by executing only {len(executor.quantum_results)} unique circuits.")

200 noisy expectation values efficiently evaluated by executing only 13 unique circuits.


Now estimate the ideal expecation value as an average of the noisy auxiliary expectation values, after scaling them by the corresponding sampled_signs and by the one_norm coefficient (obtained in the previous section)

In [8]:
# Scale noisy results by one-norm coefficient and by sampled signs
unbiased_samples = [
  one_norm * value * sign for value, sign in zip(noisy_expecation_values, sampled_signs)
]

# Estimate ideal expectation value
pec_value = sum(unbiased_samples) / len(unbiased_samples)

unmitigated_value = executor.evaluate(circuit)[0]

print(f"Expectation value without error mitigation:    {unmitigated_value:.5f}")
print(f"Expectation value with error mitigation (PEC): {pec_value:.5f}")

Expectation value without error mitigation:    0.95516
Expectation value with error mitigation (PEC): 0.99171


Check if you can obtain a better PEC estimation (with reduced statistical fluctuations)- hint: increase the number of probabilistically generated circuits. 

## Step 3: Compare with top-level PEC workflow

We will use the same circuit, noise model, representations, and executor as above, and call `execute_with_pec` in one line:

In [9]:
pec_value_top_level = mitiq.pec.execute_with_pec(circuit, executor, representations=reps, num_samples=200, random_state=30)
print(f"Expectation value with error mitigation (PEC): {pec_value_top_level:.5f}")

Expectation value with error mitigation (PEC): 0.99171
